# 📰 ETL de Notícias — Projeto Web Unifor ETL

## 🎯 Objetivo
Este notebook implementa a etapa de **extração, transformação e carga (ETL)** de notícias relacionadas ao **dólar americano** em diferentes portais de mídia.  
O objetivo é construir uma base analítica consolidada em **DuckDB**, contendo notícias coletadas de múltiplas fontes jornalísticas para posterior análise de tendências, correlação com indicadores econômicos e variação cambial.

---

## 🌐 Fontes de Dados
As notícias são coletadas diretamente de portais jornalísticos oficiais, utilizando técnicas de **Web Scraping com Selenium e BeautifulSoup**:

| Fonte | URL |
|--------|-----|
| **G1** | [https://g1.globo.com/economia/dolar/](https://g1.globo.com/economia/dolar/) |
| **CNN Brasil** | [https://www.cnnbrasil.com.br/tudo-sobre/dolar/](https://www.cnnbrasil.com.br/tudo-sobre/dolar/) |
| **Folha de S. Paulo** | [https://www1.folha.uol.com.br/folha-topicos/dolar/](https://www1.folha.uol.com.br/folha-topicos/dolar/) |

Cada fonte é processada de forma independente, com adaptação do seletor CSS conforme a estrutura HTML de cada portal.

---

## ⚙️ Estrutura do Pipeline

1. **Coleta:**  
   - Navegação automatizada com Selenium (rolagem e múltiplas páginas).  
   - Captura de títulos, datas, links e imagens.  
   - Conversão de datas e padronização de campos.

2. **Transformação:**  
   - Normalização de colunas (`dataPublicacao`, `dataExtracao`).  
   - Preenchimento de listas com tamanhos diferentes (`pad`).  
   - Remoção de duplicatas via hash composto (`titulo` + `fonte`).

3. **Carga:**  
   - Salvamento dos dados no banco analítico **DuckDB (`dados_dolar.duckdb`)**.  
   - Criação da tabela `noticias` com schema padronizado.  
   - Exportação adicional em `.csv` para interoperabilidade.

---

## 🧱 Estrutura da Tabela `noticias`

| Coluna | Tipo | Descrição |
|--------|------|-----------|
| `urlImagem` | TEXT | URL da imagem destacada da notícia |
| `dataPublicacao` | TIMESTAMP | Data de publicação extraída da página |
| `titulo` | TEXT | Título da notícia |
| `link` | TEXT | URL da notícia |
| `fonte` | TEXT | Nome do portal de origem |
| `dataExtracao` | TIMESTAMP | Data e hora da extração do dado |
| `hash` | TEXT | Identificador único (hash de `titulo` + `fonte`) |

---

## 🧾 Logs e Monitoramento
Cada etapa do notebook possui **logs estruturados** com:
- Status de execução (INFO, WARNING, ERROR).  
- Tempo total de cada etapa (G1, CNN, Folha, ETL e DB).  
- Quantidade de registros coletados por fonte.  
- Registro automático em `logs/noticias.log`.

Os logs permitem reexecutar e diagnosticar o ETL de forma auditável e transparente.

---

## 🧮 Reprodutibilidade
- Ambiente configurado com bibliotecas listadas em `requirements.txt`.  
- Notebook executável de ponta a ponta, sem dependências externas.  
- Saídas persistentes no arquivo `dados_dolar.duckdb` e export em `exports/noticias.csv`.  
- Suporte a reexecução automatizada (opcional) via `papermill` ou `nbconvert`.

---

📅 **Última atualização:** 25/10/2025
👨‍💻 **Autor:** ANDERSON DE OLIVEIRA SILVA — Projeto Web Unifor ETL (2025)


In [1]:
# =========================
# LOGGING SETUP
# =========================
import os
import logging
from logging.handlers import RotatingFileHandler
from datetime import datetime

LOG_DIR = "logs"
os.makedirs(LOG_DIR, exist_ok=True)

def get_logger(name="noticias", level=logging.INFO):
    logger = logging.getLogger(name)
    logger.setLevel(level)
    if logger.handlers:
        return logger  # evita múltiplos handlers no notebook

    # Formato com campos extras opcionais (fonte, etapa)
    fmt = "%(asctime)s | %(levelname)s | %(name)s | %(message)s"
    datefmt = "%Y-%m-%d %H:%M:%S"

    file_handler = RotatingFileHandler(
        os.path.join(LOG_DIR, "noticias.log"),
        maxBytes=2_000_000,
        backupCount=5,
        encoding="utf-8"
    )
    file_handler.setLevel(level)
    file_handler.setFormatter(logging.Formatter(fmt, datefmt=datefmt))

    console = logging.StreamHandler()
    console.setLevel(level)
    console.setFormatter(logging.Formatter(fmt, datefmt=datefmt))

    logger.addHandler(file_handler)
    logger.addHandler(console)
    return logger

log = get_logger()
log.info("===== INÍCIO EXECUÇÃO noticias.ipynb =====")

2025-10-25 14:18:06 | INFO | noticias | ===== INÍCIO EXECUÇÃO noticias.ipynb =====


In [2]:
# =========================
# IMPORTS & FONTES
# =========================
import time
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium import webdriver

# Fontes de Notícias
fonte_g1   = "https://g1.globo.com/economia/dolar/"
fonte_cnn  = "https://www.cnnbrasil.com.br/tudo-sobre/dolar/"
fonte_folha= "https://www1.folha.uol.com.br/folha-topicos/dolar/"

# Utils
def agora():
    return datetime.now().strftime("%d/%m/%Y %H:%M:%S")

def pad(lst, n, fill=""):
    return lst + [fill] * max(0, n - len(lst))

def safe_df_from_lists(fonte, url_imagem, data_publicacao, titulos, links):
    """
    Garante que todas as colunas tenham o mesmo tamanho (com pad) e loga diferenças.
    """
    n_max = max(len(url_imagem), len(data_publicacao), len(titulos), len(links))
    if len({len(url_imagem), len(data_publicacao), len(titulos), len(links)}) != 1:
        log.warning(f"[{fonte}] Listas com tamanhos diferentes: "
                    f"img={len(url_imagem)} | data={len(data_publicacao)} | tit={len(titulos)} | link={len(links)}. Fazendo pad para {n_max}")
    df = pd.DataFrame({
        "urlImagem":      pad(url_imagem, n_max, ""),
        "dataPublicacao": pad(data_publicacao, n_max, ""),
        "titulo":         pad(titulos, n_max, ""),
        "link":           pad(links, n_max, ""),
        "fonte":          [fonte]*n_max,
        "dataExtracao":   [agora()]*n_max,
    })
    # filtra linhas totalmente vazias
    df = df[~(df["titulo"].eq("") & df["link"].eq("") & df["urlImagem"].eq(""))]
    return df

def nova_sessao_chrome():
    options = webdriver.ChromeOptions()
    # exemplo: options.add_argument("--headless=new")
    driver = webdriver.Chrome(options=options)
    return driver


In [3]:
# =========================
# G1
# =========================
inicio = time.perf_counter()
log.info("[G1] Iniciando extração")

driver = None
df_noticias = pd.DataFrame()

try:
    driver = nova_sessao_chrome()
    driver.get(fonte_g1)
    time.sleep(5)
    try:
        driver.find_element(By.CSS_SELECTOR, "svg.fc-cancel-icon-svg").click()
        log.info("[G1] Pop-up fechado")
    except Exception:
        log.info("[G1] Pop-up não encontrado (ok)")

    # Scroll p/ carregar mais
    for _ in range(10):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)

    html_g1 = driver.page_source
    soup = BeautifulSoup(html_g1, "html.parser")

    url_imagem = [img.get("src","") for img in soup.select("img.bstn-fd-picture-image")]
    titulos    = [t.get_text(strip=True) for t in soup.select("div.feed-post-body-title a")]
    links      = [a.get("href","") for a in soup.select("a.feed-post-link")]

    # Deriva data da URL (se falhar, deixa vazio)
    data_publicacao = []
    for link in links:
        try:
            partes = link.split("/")
            data_publicacao.append(f"{partes[7]}/{partes[6]}/{partes[5]}")
        except Exception:
            data_publicacao.append("")

    df_g1 = safe_df_from_lists("G1", url_imagem, data_publicacao, titulos, links)
    df_noticias = df_g1.copy()

    log.info(f"[G1] Coletados {len(df_g1)} registros")
except Exception as e:
    log.exception(f"[G1] ERRO na extração: {e}")
finally:
    if driver:
        driver.quit()
        log.info("[G1] Driver encerrado")

log.info(f"[G1] Concluído em {time.perf_counter()-inicio:.2f}s")


2025-10-25 14:18:30 | INFO | noticias | [G1] Iniciando extração
2025-10-25 14:18:54 | INFO | noticias | [G1] Pop-up fechado
2025-10-25 14:19:16 | INFO | noticias | [G1] Coletados 32 registros
2025-10-25 14:19:18 | INFO | noticias | [G1] Driver encerrado
2025-10-25 14:19:18 | INFO | noticias | [G1] Concluído em 47.86s


In [4]:
# =========================
# CNN
# =========================
inicio = time.perf_counter()
log.info("[CNN] Iniciando extração")

driver = None
try:
    driver = nova_sessao_chrome()
    driver.get(fonte_cnn)
    time.sleep(5)

    try:
        wait = WebDriverWait(driver, 10)
        botao_ok = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".btn-agree button")))
        botao_ok.click()
        log.info("[CNN] Aceite de cookies clicado")
    except TimeoutException:
        log.info("[CNN] Botão de cookies não apareceu (ok)")

    total_regs = 0

    for pagina in range(2):  # 2 páginas
        time.sleep(3)
        html_cnn = driver.page_source
        soup = BeautifulSoup(html_cnn, "html.parser")

        container = soup.select_one('ul[data-section="article_list"]')

        url_imagem = []
        if container:
            url_imagem = [img.get("src","") for img in container.select("img") if img.get("src")]

        titulos = [t.get_text(strip=True) for t in soup.select("div.flex.flex-col.gap-4 h2")]
        links   = [a.get("href","") for a in soup.select("div.flex.flex-col.gap-4 a")]
        # Remove link de "ao vivo"
        links   = [u for u in links if u != 'https://www.cnnbrasil.com.br/ao-vivo/']

        data_publicacao = []
        for tag in soup.select("time"):
            raw = tag.get("datetime")
            if raw:
                try:
                    data_publicacao.append(datetime.fromisoformat(raw).strftime("%d/%m/%Y"))
                except Exception:
                    data_publicacao.append("")

        df_cnn = safe_df_from_lists("CNN", url_imagem, data_publicacao, titulos, links)
        total_regs += len(df_cnn)
        df_noticias = pd.concat([df_noticias, df_cnn], ignore_index=True)

        log.info(f"[CNN] Página {pagina+1}: {len(df_cnn)} registros")

        # Tenta ir para próxima página (se existir)
        try:
            driver.find_element(By.CSS_SELECTOR, "a[aria-label='Ir para próxima página']").click()
            log.info("[CNN] Próxima página clicada")
        except Exception:
            log.info("[CNN] Próxima página não disponível; encerrando paginação")
            break

    log.info(f"[CNN] Total coletado: {total_regs}")
except Exception as e:
    log.exception(f"[CNN] ERRO na extração: {e}")
finally:
    if driver:
        driver.quit()
        log.info("[CNN] Driver encerrado")

log.info(f"[CNN] Concluído em {time.perf_counter()-inicio:.2f}s")


2025-10-25 14:19:44 | INFO | noticias | [CNN] Iniciando extração
2025-10-25 14:19:55 | INFO | noticias | [CNN] Aceite de cookies clicado
2025-10-25 14:19:58 | INFO | noticias | [CNN] Página 1: 10 registros
2025-10-25 14:20:01 | INFO | noticias | [CNN] Próxima página clicada
2025-10-25 14:20:04 | INFO | noticias | [CNN] Página 2: 10 registros
2025-10-25 14:20:06 | INFO | noticias | [CNN] Próxima página clicada
2025-10-25 14:20:06 | INFO | noticias | [CNN] Total coletado: 20
2025-10-25 14:20:09 | INFO | noticias | [CNN] Driver encerrado
2025-10-25 14:20:09 | INFO | noticias | [CNN] Concluído em 25.32s


In [5]:
# =========================
# FOLHA
# =========================
inicio = time.perf_counter()
log.info("[Folha] Iniciando extração")

driver = None
try:
    driver = nova_sessao_chrome()
    driver.get(fonte_folha)
    wait = WebDriverWait(driver, 20)

    for rodada in range(4):
        try:
            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "main .row, main article, main li")))
            candidatos = driver.find_elements(By.CSS_SELECTOR, "button.c-button.c-button--expand")
            if candidatos:
                botao = candidatos[0]
                driver.execute_script("arguments[0].scrollIntoView({block:'center'});", botao)
                time.sleep(0.5)
                try:
                    botao.click()
                    log.info(f"[Folha] 'Mostrar mais' clicado (rodada {rodada+1})")
                except Exception:
                    driver.execute_script("arguments[0].click();", botao)
                    log.info(f"[Folha] 'Mostrar mais' clicado via JS (rodada {rodada+1})")
            else:
                log.info("[Folha] Botão 'Mostrar mais' não encontrado; parando")
                break
        except TimeoutException:
            log.info("[Folha] Timeout aguardando conteúdo; seguindo")
        finally:
            time.sleep(1.5)

    html_folha = driver.page_source
    soup = BeautifulSoup(html_folha, "html.parser")

    url_imagem = [img.get("data-src","") for img in soup.select("img.c-headline__image") if img.get("data-src")]
    titulos    = [t.get_text(strip=True) for t in soup.select("h2.c-headline__title")]
    links      = [a.get("href","") for a in soup.select("a.c-headline__url")]

    data_publicacao_raw = [t.get("datetime","") for t in soup.select("time.c-headline__dateline[itemprop='datePublished']")]
    data_publicacao = []
    for d in data_publicacao_raw:
        if d:
            try:
                data_publicacao.append(datetime.strptime(d, "%Y-%m-%d %H:%M:%S").strftime("%d/%m/%Y"))
            except Exception:
                # tente outro formato comum
                try:
                    data_publicacao.append(datetime.fromisoformat(d).strftime("%d/%m/%Y"))
                except Exception:
                    data_publicacao.append("")

    df_folha = safe_df_from_lists("Folha", url_imagem, data_publicacao, titulos, links)
    # opcional: manter apenas linhas com imagem
    df_folha = df_folha[df_folha["urlImagem"] != ""]
    df_noticias = pd.concat([df_noticias, df_folha], ignore_index=True)

    log.info(f"[Folha] Coletados {len(df_folha)} registros")
except Exception as e:
    log.exception(f"[Folha] ERRO na extração: {e}")
finally:
    if driver:
        driver.quit()
        log.info("[Folha] Driver encerrado")

log.info(f"[Folha] Concluído em {time.perf_counter()-inicio:.2f}s")


2025-10-25 14:20:31 | INFO | noticias | [Folha] Iniciando extração
2025-10-25 14:20:43 | INFO | noticias | [Folha] 'Mostrar mais' clicado (rodada 1)
2025-10-25 14:20:45 | INFO | noticias | [Folha] 'Mostrar mais' clicado (rodada 2)
2025-10-25 14:20:48 | INFO | noticias | [Folha] 'Mostrar mais' clicado (rodada 3)
2025-10-25 14:20:50 | INFO | noticias | [Folha] 'Mostrar mais' clicado (rodada 4)
2025-10-25 14:20:53 | WARNING | noticias | [Folha] Listas com tamanhos diferentes: img=99 | data=99 | tit=103 | link=103. Fazendo pad para 103
2025-10-25 14:20:53 | INFO | noticias | [Folha] Coletados 99 registros
2025-10-25 14:20:55 | INFO | noticias | [Folha] Driver encerrado
2025-10-25 14:20:55 | INFO | noticias | [Folha] Concluído em 24.45s


In [6]:
# =========================
# NORMALIZAÇÃO & DEDUPE
# =========================
inicio = time.perf_counter()
log.info("[ETL] Normalizando e removendo duplicatas")

df = df_noticias.copy()

# Datas
for col in ["dataPublicacao", "dataExtracao"]:
    if col in df.columns:
        df[col] = pd.to_datetime(df[col], dayfirst=True, errors="coerce")

# Hash para dedupe por título+fonte
df["hash"] = (df["titulo"].fillna("") + "|" + df["fonte"].fillna("")).apply(lambda x: str(hash(x)))
antes = len(df)
df = df.drop_duplicates(subset=["hash"])
apos = len(df)

log.info(f"[ETL] Linhas antes: {antes} | após dedupe: {apos} | removidas: {antes-apos}")
log.info("[ETL] Amostra:")
log.info(df.head(3).to_string(index=False))

log.info(f"[ETL] Concluído em {time.perf_counter()-inicio:.2f}s")


2025-10-25 14:21:07 | INFO | noticias | [ETL] Normalizando e removendo duplicatas
2025-10-25 14:21:07 | INFO | noticias | [ETL] Linhas antes: 151 | após dedupe: 151 | removidas: 0
2025-10-25 14:21:07 | INFO | noticias | [ETL] Amostra:
2025-10-25 14:21:07 | INFO | noticias |                                                                                                                                                                                                                                                        urlImagem dataPublicacao                                                                                               titulo                                                                  link fonte        dataExtracao                 hash
                         https://s2-g1.glbimg.com/5WrVoTrL2c24Do4OEg3tDprvptE=/540x304/top/smart/https://i.s3.glbimg.com/v1/AUTH_59edd422c0c84a879bd37670ae4f538a/internal_photos/bs/2024/1/P/tCWnbnTHuFIxLcWlmcGA/globo-canal-5-20241101-1

In [7]:
# =========================
# DUCKDB
# =========================
inicio = time.perf_counter()
log.info("[DB] Iniciando persistência em DuckDB")

import duckdb
import os

DB_PATH = "dados_dolar.duckdb"
TABLE   = "noticias"

os.makedirs("exports", exist_ok=True)

con = None
try:
    con = duckdb.connect(DB_PATH)
    con.execute(f"""
        CREATE TABLE IF NOT EXISTS {TABLE} (
            urlImagem       TEXT,
            dataPublicacao  TIMESTAMP,
            titulo          TEXT,
            link            TEXT,
            fonte           TEXT,
            dataExtracao    TIMESTAMP,
            hash            TEXT
        );
    """)
    log.info("[DB] Tabela verificada/criada")

    # registra DF e insere
    con.register("df_stage", df)
    inseridos = con.execute(f"""
        INSERT INTO {TABLE}
        SELECT urlImagem, dataPublicacao, titulo, link, fonte, dataExtracao, hash
        FROM df_stage;
    """).rowcount
    log.info(f"[DB] Inseridos {inseridos if inseridos is not None else 0} registros")

    # export CSV a partir do DF (ou da tabela, se preferir)
    df.to_csv("./exports/noticias.csv", index=False, encoding="utf-8")
    log.info("[DB] Export gerado em ./exports/noticias.csv")

except Exception as e:
    log.exception(f"[DB] ERRO na persistência: {e}")
finally:
    if con:
        con.close()
        log.info("[DB] Conexão encerrada")

log.info(f"[DB] Concluído em {time.perf_counter()-inicio:.2f}s")
log.info("===== FIM EXECUÇÃO noticias.ipynb =====")


2025-10-25 14:21:19 | INFO | noticias | [DB] Iniciando persistência em DuckDB
2025-10-25 14:21:19 | INFO | noticias | [DB] Tabela verificada/criada
2025-10-25 14:21:19 | INFO | noticias | [DB] Inseridos -1 registros
2025-10-25 14:21:19 | INFO | noticias | [DB] Export gerado em ./exports/noticias.csv
2025-10-25 14:21:19 | INFO | noticias | [DB] Conexão encerrada
2025-10-25 14:21:20 | INFO | noticias | [DB] Concluído em 0.36s
2025-10-25 14:21:20 | INFO | noticias | ===== FIM EXECUÇÃO noticias.ipynb =====
